In [ ]:
!pip install pypdf langchain docarray pydantic==1.10.8 torch transformers llama-cpp-python
!pip install instructorembedding langchain_community chromadb
!huggingface-cli login

import torch
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `pantelis` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `pantelis`

In [3]:
# Load LLaMA-2 model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Load and split PDF
pdf_path = "/content/...pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(pages)

In [ ]:
# Initialize embeddings 
instructor_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-miniLM-L6-v2")

# Chroma vector database
db_directory = "db"
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=db_directory)

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Prompt template
template = """
### Instruction:
You are an AI assistant. Answer the question based on the context below. If you can't answer the question, reply "I don't know".

### Context:
{context}

### Question:
{question}

### Answer:
"""
prompt = PromptTemplate.from_template(template)

#parser = StrOutputParser()

In [ ]:
def generate_answer(question):
    print(f"\nQuestion: {question}")

    # Retrieve relevant documents
    retrieved_docs = retriever.get_relevant_documents(question)

    retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Create formatted prompt
    formatted_prompt = prompt.format(context=retrieved_text, question=question)

    # Tokenize input for the model
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Generate model response
    with torch.no_grad():
        output = model.generate(**inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response_text = response_text.replace(formatted_prompt, "").strip()

    print(f"Answer: {response_text}")
    return response_text

In [ ]:
questions = [
    "What is ADMET?",
    "How does deep learning contribute to drug discovery?",
    "What are the challenges of ADMET modeling?"
]

for question in questions:
    generate_answer(question)


Question: What is ADMET?


<ipython-input-9-25a44ee58f75>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


Answer: ADMET stands for Absorption, Distribution, Metabolism, Excretion, and Toxicity. It refers to the study of the pharmacokinetic and pharmacodynamic properties of a drug, including how it is absorbed, distributed, metabolized, and excreted, as well as its potential toxicity.

Question: How does deep learning contribute to drug discovery?
Answer: Deep learning contributes to drug discovery by enabling the accurate prediction of drug-target interactions and binding affinity through the use of universal recurrent and convolutional neural networks. This allows for the identification of therapeutically active molecules with much greater efficiency and accuracy than traditional methods. Additionally, deep learning tools can be used for advancing drug discovery and development by predicting drug-target interactions and binding affinity, and identifying potential drug targets for specific diseases.

Question: What are the challenges of ADMET modeling?
